In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import skimage.io as skio
import skimage
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import tifffile
import numpy as np
from utils import *
import os
from pathlib import Path
import trackpy as tp
import scipy
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from scipy.stats import anderson
import glob
import matplotlib.animation as animation
import matplotlib.colors as mcolors
import warnings
from matplotlib.patches import Rectangle

In [ ]:
fit_method = "lq"
box_side_length = 7
drift = 0
min_gradient = 800
px_to_nm = 72
frame_to_s = 0.25

min_fibre_size = 60
max_link_displacement_px = 2
max_gap = 0
min_tray_length = 3

#customize the scalebar on the video
scale_bar_length = 1
scale_bar_pixels = int(scale_bar_length / (px_to_nm/1000)) 
range_min = 0
range_max = 200 

In [ ]:
file_directory = r"C:\Users\lizau\Desktop\walker_tracker_for_article\walker_tracker\example"
in_files = glob.glob(f'{file_directory}/*.tif')

In [ ]:
# This cell: 
#   identifies walkers on the track
#   localizes the walkers
#   calculates the walkers' steps and trajectories
#   fits a 2D Gaussian distribution to the walkers' steps
#   plots the 2D distribution
#   calculates the diffusion coefficient in each dimension, average dwell time and the time the walker would need to cover the length of 1 micrometre

diff_coeff_e1 = []
diff_coeff_e2 = []
time_1000_nm = []
dwell_t = []
file_basename = []
for file in in_files:
    basename = os.path.basename(file)
    file_basename += [basename]
    basename_noext, ext = os.path.splitext(basename)
    out_dir = Path(f'{file_directory}\out_fit_2D_Gaussian')
    os.makedirs(out_dir, exist_ok=True)
    take_only_walkers_on_fibre_trajectory(file, out_dir / basename_noext, min_fibre_size=min_fibre_size)
    new_out_locs = fit_single_molecules(
        out_dir,
        basename,
        fit_method=fit_method,
        box_side_length=box_side_length,
        drift=drift,
        min_gradient=min_gradient,
        px_to_nm=px_to_nm,
    )
    locs = pd.read_hdf(new_out_locs, "locs")
    locs["mass"] = locs.photons
    
    tray = tp.link(locs, max_link_displacement_px)#, memory=max_gap) 
    
    # Optional: if you want to rotate the data (doesn't change the fit)
    '''
    theta_deg = 180
    theta = np.radians(theta_deg)
    
    # Rotation matrix
    R = np.array([[np.cos(theta), -np.sin(theta)],
                [np.sin(theta),  np.cos(theta)]])

    # Apply rotation
    xy = tray[["x", "y"]].values
    rotated_xy = xy @ R.T   # matrix multiply

    tray[["x", "y"]] = rotated_xy
    '''
    # Count the length of trajectories
    tray_by_particle = tray.groupby(["particle"])
    tray["length"] = tray_by_particle["particle"].transform("count")
    dwell_time = tray["length"].mean()

    # Exclude very short trays
    tray = tray.query(f"length>={min_tray_length}")

    # Calculate steps
    steps = tray.groupby(["particle"]).apply(get_steps_from_df)
    steps["step_len"] = np.sqrt(steps.dx**2 + steps.dy**2)

    # Save trajectories and steps
    tray_out = Path(f'{out_dir}/{basename_noext}_link{max_link_displacement_px}_traylen{min_tray_length}.tray.csv')
    tray.to_csv(tray_out)
    steps_out = Path(f'{out_dir}/{basename_noext}.steps.csv')
    steps.to_csv(steps_out)

    #distribution_check = []
    steps_matrix = steps[['dx', 'dy']].to_numpy()
    num_columns = steps_matrix.shape[1]  
    normal_str = ''
    for var_idx in range(num_columns):
        anderson_data = anderson(steps_matrix[:, var_idx])
        is_normal = anderson_data.statistic < anderson_data.critical_values[1]
        #distribution_check += [is_normal]
        if not is_normal:
            normal = f"variable {f'dx' if var_idx == 0 else f'dy'} NOT normally distributed at {anderson_data.significance_level[1]}%\n"
        else:
            normal = f"variable {f'dx' if var_idx == 0 else f'dy'} normally distributed at {anderson_data.significance_level[1]}%\n"
        normal_str += normal
    #print(normal_str) 

    # Make a walker movie with drawing the trajectories

    #walker_traj_movie_BW(tray, out_dir, basename_noext, range_max=50)

    # Create a 2D histogram of step sizes
    x = steps['dx']
    y = steps['dy']
    bins = (75, 75)
    hist, xedges, yedges = np.histogram2d(x, y, bins=bins, density=True)
    xcent = (xedges[1:] + xedges[:-1])/2
    ycent = (yedges[1:] + yedges[:-1])/2
    x_grid, y_grid = np.meshgrid(xcent, ycent)
    xy = np.stack((x_grid.ravel(), y_grid.ravel())).T

    # Fit a 2D gaussian to the histogram
    initial_guess = [0, 0, 1, 1, 0]
    try:
        popt, _ = curve_fit(gaussian_2d, xy, hist.T.ravel(), initial_guess)
    except RuntimeError as e:
        warnings.warn(f"Optimal parameters not found: {e}")
        popt = None 
    if popt is not None:
        mu = [popt[0], popt[1]]
        cov_matrix = [[popt[2], popt[4]], [popt[4], popt[3]]]
        e1, e2, rot = cov_to_axes_and_rotation(cov_matrix)

    # For saving the info about diffusion
    info = {}
    diff_info_out = Path(f'{out_dir}/{basename}.diff')
    info["average dwell time"] = float(dwell_time*frame_to_s)
    if popt is not None:
        info["diff_long_nm_nm_s"] = float(e1*px_to_nm*px_to_nm/(2*frame_to_s))
        info["diff_short_nm_nm_s"] = float(e2*px_to_nm*px_to_nm/(2*frame_to_s))
        info["time_per_1000nm_s_long_axis"] = 1000 * 1000 / info["diff_long_nm_nm_s"] / 2
        info["mean_step_nm_s"] = float(np.sqrt(2 * info["diff_long_nm_nm_s"] * frame_to_s) / frame_to_s)
        info["normality_str"] = normal_str
        info["anderson_test"] = anderson_data
        info["mu_x"] = float(round(mu[0], 3))
        info["mu_y"] = float(round(mu[1], 3))
        info["sigma_x"] = float(round(cov_matrix[0][0], 3))
        info["sigma_y"] = float(round(cov_matrix[1][1], 3))
        info["sigma_xy"] = float(round(cov_matrix[0][1], 3))

        diff_coeff_e1 += [info["diff_long_nm_nm_s"]]
        diff_coeff_e2 += [info["diff_short_nm_nm_s"]]
        time_1000_nm += [info["time_per_1000nm_s_long_axis"]]
        dwell_t += [info["average dwell time"]]

        # Calculate the probability density function
        z = gaussian_2d(xy, *popt)
    if popt is None:
        diff_coeff_e1 += [np.nan]
        diff_coeff_e2 += [np.nan]
        time_1000_nm += [np.nan]

    write_yaml(info, diff_info_out)

    
# --- Plot trajectory lengths histogram ---
    fig, ax = plt.subplots(figsize=(6,6))
    plt.sca(ax)
    tray.length.hist()
    plt.xlabel("Trajectory Length")
    plt.title("Histogram of trajectory steps")
    fig.savefig(f'{out_dir}/{basename}_traj_len_hist.png')
    plt.close(fig)

# --- Plot 2D Gaussian fit ---
    # --- Histogram in Blues (NO colorbar) ---
    fig, ax = plt.subplots(figsize=(6,6))
    ax.hist2d(
        x*72, -y*72,
        bins=bins,
        range=[(-100,100), (-100,100)],
        density=True,
        cmap="Blues"
    )

    # --- Contour in Greys (WITH colorbar) ---
    # Filled contours (blocky discrete colors, not thin lines)
    csf = ax.contourf(
    x_grid*72, -y_grid*72, z.reshape(x_grid.shape),
    cmap="Greys"
    )
    for coll in csf.collections:
        coll.set_visible(False)  # remove filled patches from the axes

    # Draw only contour lines (visible)
    cs = ax.contour(
        x_grid*72, -y_grid*72, z.reshape(x_grid.shape),
        cmap="Greys", 
        linewidths=2
    )

    # Use contourf object for colorbar
    cbar = fig.colorbar(csf, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Step size probability density", fontsize=18)
    cbar.ax.tick_params(labelsize=18, width=2)
    cbar.outline.set_linewidth(2)

    # --- Axes formatting ---
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel('dx [nm]', fontsize = 18)
    plt.ylabel('dy [nm]', fontsize = 18)
    plt.xticks(np.arange(-100, 101, 50))
    plt.yticks(np.arange(-100, 101, 50))  
    #plt.xticks(rotation=30)   # rotate x-axis tick labels by 45°
    #plt.yticks(rotation=30)   # rotate y-axis tick labels by 45°
    plt.tick_params(axis='both', labelsize=18)
    plt.close()
    fig.savefig(f'{out_dir}/{basename}_2D_fit.png', bbox_inches='tight')


diff_params = {'BASENAME': file_basename, 'e1': diff_coeff_e1, 'e2': diff_coeff_e2, 'time_1000_nm': time_1000_nm, 'average_dwell_time': dwell_t}
diff_params_df = pd.DataFrame(diff_params)
diff_params_df.to_csv(f'{out_dir}/diff_info.csv')